In [1]:
from sys import monitoring

In [2]:
depth = 0
def on_start(code, offset):
    global depth
    depth += 1
    prefix = depth * ">"
    print(f"{prefix} {code.co_name}")

def on_return(code, offset, retval):
    global depth
    prefix = depth * "<"
    print(f"{prefix} {code.co_name}, return: {retval}")
    depth -= 1
ID = 3
monitoring.use_tool_id(ID, "tracer")
monitoring.register_callback(ID, monitoring.events.PY_START, on_start)
monitoring.register_callback(ID, monitoring.events.PY_RETURN, on_return)

def trace(func):
    def inner(*args, **kwargs):
        monitoring.set_local_events(ID, func.__code__, monitoring.events.PY_START|monitoring.events.PY_RETURN)
        ret = func(*args, **kwargs)
        return ret
    return inner

@trace
def fib(i):
    if i <= 1:
        return 1
    else:
        return fib(i-1) + fib(i-2)

fib(10)
monitoring.free_tool_id(ID)


> fib
>> fib
>>> fib
>>>> fib
>>>>> fib
>>>>>> fib
>>>>>>> fib
>>>>>>>> fib
>>>>>>>>> fib
>>>>>>>>>> fib
<<<<<<<<<< fib, return: 1
>>>>>>>>>> fib
<<<<<<<<<< fib, return: 1
<<<<<<<<< fib, return: 2
>>>>>>>>> fib
<<<<<<<<< fib, return: 1
<<<<<<<< fib, return: 3
>>>>>>>> fib
>>>>>>>>> fib
<<<<<<<<< fib, return: 1
>>>>>>>>> fib
<<<<<<<<< fib, return: 1
<<<<<<<< fib, return: 2
<<<<<<< fib, return: 5
>>>>>>> fib
>>>>>>>> fib
>>>>>>>>> fib
<<<<<<<<< fib, return: 1
>>>>>>>>> fib
<<<<<<<<< fib, return: 1
<<<<<<<< fib, return: 2
>>>>>>>> fib
<<<<<<<< fib, return: 1
<<<<<<< fib, return: 3
<<<<<< fib, return: 8
>>>>>> fib
>>>>>>> fib
>>>>>>>> fib
>>>>>>>>> fib
<<<<<<<<< fib, return: 1
>>>>>>>>> fib
<<<<<<<<< fib, return: 1
<<<<<<<< fib, return: 2
>>>>>>>> fib
<<<<<<<< fib, return: 1
<<<<<<< fib, return: 3
>>>>>>> fib
>>>>>>>> fib
<<<<<<<< fib, return: 1
>>>>>>>> fib
<<<<<<<< fib, return: 1
<<<<<<< fib, return: 2
<<<<<< fib, return: 5
<<<<< fib, return: 13
>>>>> fib
>>>>>> fib
>>>>>>> fib
>>>>>>>> 

In [3]:
!pip install libcst


[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import libcst

In [5]:
class FuncIntVisitor(libcst.CSTVisitor):
    def __init__(self):
        self.current_function = None
        self.func_int = dict()
        
    def visit_FunctionDef(self, node: libcst.FunctionDef):
        self.current_function = node.name.value
        if self.current_function not in self.func_int:
            self.func_int[self.current_function] = list()
    def leave_FunctionDef(self, node: libcst.FunctionDef):
        self.current_function = None

    def visit_Integer(self, node: libcst.Integer):
        if self.current_function:
            self.func_int[self.current_function].append(node.value)
            

In [6]:
m = libcst.parse_module("""
def foo(bar):
    buz = 3
    return bar + buz +1

foo(3)
print('123')""")

In [7]:
v = FuncIntVisitor()
m.visit(v)
print(v.func_int)

{'foo': ['3', '1']}


In [8]:
class IntTransformer(libcst.CSTTransformer):
    def leave_Integer(self, node: libcst.Integer, updated: libcst.Integer):
        value = str(int(updated.value) + 1)
        return updated.with_changes(value=value)

In [9]:
m = libcst.parse_module("""
def foo(bar):
    return bar + 1

foo(3)
print('123')""")

In [10]:
t = IntTransformer()

In [11]:
n = m.visit(t)

In [12]:
print(n.code)


def foo(bar):
    return bar + 2

foo(4)
print('123')
